# Preston's try

In [1]:
import pandas as pd
from pandas import Series
import numpy as np
import json

## Preparation

In [2]:
# load parameters
params_df = pd.read_csv("ML_param.csv")
# Just doing what the repo did
del params_df['coord_dec'] 
del params_df['coord_ra']
del params_df['id']
del params_df['parent']

In [11]:
# load classfication data
data_df = pd.read_csv("../classifications/lsst_run_one.csv")
data_df.head()
classify_df = data_df.loc[data_df['workflow_name'] == "Difference Imaging Classifier"]

image_class = np.zeros(classify_df.shape[0], dtype='int, object')
idx = 0
for _, row in classify_df.iterrows():
    s_data = json.loads(row.subject_data) #Subject Data
    s_data = s_data.get(list(s_data.keys())[0])
    # cut "/home/......./cutout" and ".png"
    image_num = s_data.get(list(s_data.keys())[1])[47:-4]
    # annotations
    a_data = json.loads(row.annotations)[0]
    classification = a_data['value']
    image_class[idx] = (int(image_num), classification)
    idx += 1

In [41]:
# add a new classification column, init with None
params_df = params_df.assign(Classification=Series(np.full(params_df.shape[0], None)))
for image in image_class:
    params_df.loc[image[0], "Classification"] = image[1]
# drop the rows with no classification
df = params_df[params_df["Classification"].notnull()]

In [45]:
df

,image,base_CircularApertureFlux_70_0_flux,base_CircularApertureFlux_9_0_fluxSigma,base_CircularApertureFlux_6_0_flux,ip_diffim_PsfDipoleFlux_neg_centroid_ySigma,base_PixelFlags_flag_interpolated,ip_diffim_DipoleFit_flag_edge,base_PsfFlux_flag_edge,ip_diffim_PsfDipoleFlux_neg_flag,base_PixelFlags_flag,...,base_CircularApertureFlux_3_0_flag,base_SdssShape_flag,base_CircularApertureFlux_35_0_flag,ip_diffim_DipoleFit_flag_classification,base_PixelFlags_flag_saturatedCenter,base_PixelFlags_flag_bad,base_CircularApertureFlux_12_0_flux,ip_diffim_DipoleFit_orientation,base_CircularApertureFlux_25_0_flux,Classification
0,0,9631.897876,2.541888e+02,714.726562,NaN,False,False,False,False,False,...,False,False,False,True,False,False,418.723064,130.761641,1011.415786,Dipole
1,1,4203.040442,2.027053e+02,1035.672363,NaN,False,False,False,False,False,...,False,False,False,False,False,False,1113.044229,NaN,1657.226911,Possible Transient
2,2,1545.866122,3.114419e+02,-148.509018,NaN,False,False,False,False,False,...,False,True,False,True,False,False,50.727865,-23.666280,194.104162,Subtraction Error
3,3,NaN,2.324338e+02,32.771393,NaN,False,False,False,False,False,...,False,False,False,False,False,False,-94.296964,NaN,538.538759,Subtraction Error
4,4,10168.914930,2.785685e+02,718.301331,NaN,False,False,False,False,False,...,False,True,False,True,False,False,988.245036,85.948590,1932.348548,Subtraction Error
5,5,2893.304278,2.452207e+02,-325.450531,NaN,False,False,False,False,False,...,False,True,False,False,False,False,-70.508982,NaN,146.085124,Dipole
6,6,10133.273388,2.326413e+02,90.175575,NaN,False,False,False,False,False,...,False,True,False,False,False,False,594.713851,NaN,2011.669773,Possible Variable Star
7,7,6990.860630,3.034678e+02,-1155.255981,NaN,False,False,False,False,False,...,False,True,False,False,False,False,-1532.151207,NaN,-1720.499918,Dipole
8,8,5884.912088,2.444883e+02,382.576508,NaN,False,False,False,False,False,...,False,False,False,False,False,False,264.164465,NaN,1423.317802,Dipole
9,9,2209.495289,2.207661e+02,-60.280769,NaN,False,False,False,False,False,...,False,True,False,False,False,False,-152.457589,NaN,-12.382492,Possible Variable Star
